<a href="https://colab.research.google.com/github/aidanjmaldonado/penny-stock-lstm/blob/main/penny_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn
!pip install matplotlib
!pip install numpy

In [2]:
import torch.nn as nn
import sklearn
import numpy as np
import pandas as pd
import sqlite3 as sql
import requests
import sys
import matplotlib.pyplot as plt



In [3]:
hidden_layers = 100
inputs = 50 #size of chunks?
hidden_size = 200 #need to tune
horizon = 20

In [4]:
class PennyStockModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size = inputs, hidden_size=hidden_size)
        self.linear = nn.Linear(hidden_size, horizon)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [5]:
# Download database from github repository
historical_url = "https://raw.githubusercontent.com/CSE-115-UCSC/penny-stock-lstm/main/historicaldata.db"
scrape_request = requests.get(historical_url)

try:
  # Contingent on request status
  scrape_request = requests.get(historical_url)
  scrape_request.raise_for_status()

  # Create local database from pull, name 'historicaldata.db'
  with open("historical.db", "wb") as db_file:
    db_file.write(scrape_request.content)

  sys.stderr.write("Request to download database succeeded")

except:
  # Report failed request status
  sys.stderr.write("Request to download database failed")

Request to download database succeeded

In [6]:
# Connect to SQlite database
try:
  db = 'historical.db'
  sqliteConnection = sql.connect(db)
  cursor = sqliteConnection.cursor()
  print(f'SQlite connected with {db}')

except:
  sys.stderr.write("Failed to connect to database")

SQlite connected with historical.db


In [10]:

# Create query finding low instances of GSAT
try:
  query = "SELECT * FROM GSAT WHERE low >= 1.29"
  cursor.execute(query)

except:
  sys.stderr.write("Failed to query database")

In [11]:
# Turn SQlite Database into Pandas Dataframe
df = pd.read_sql_query(query, sqliteConnection)
df.head()

,ticker,volume,volume_weighted_average,open,close,high,low,time,number_of_trades
0,GSAT,55076.0,1.3037,1.315,1.295,1.32,1.2947,1657557000000,110
1,GSAT,143647.0,1.303,1.2995,1.315,1.315,1.295,1657557300000,185
2,GSAT,41999.0,1.313,1.315,1.315,1.32,1.31,1657557600000,35
3,GSAT,41523.0,1.3124,1.315,1.31,1.32,1.31,1657557900000,36
4,GSAT,7979.0,1.3115,1.315,1.32,1.32,1.31,1657558200000,29
